 # TASK 2

 ### Setup
 - Import the modules: `pandas`, `triplets`, `tabulate`
 - Define helper function `format()` for cleaner print output
    - It is used where it makes sense and makes the formatting better
 - Import the data and inspect it 

In [16]:
import pandas as pd
import triplets.rdf_parser as parser
from tabulate import tabulate

# ----------------------
def format(data):
    return print(tabulate(data, headers="keys", tablefmt="simple_grid"),"\n")
# ----------------------


FILE_PATH = "20210325T1530Z_1D_NL_EQ_001.xml"
df = pd.read_RDF([FILE_PATH])
print("=================================")
print(df.head()) # Confirm data
print("=================================")
for key,val in df.types_dict().items():
    print(key,val)
print("=================================")


                                     ID    KEY                                             VALUE                           INSTANCE_ID
0  9a3aa92c-962f-43cf-852a-acf6d72a0fef   Type                                      Distribution  97c347d7-92f6-4944-9e34-738df43e9561
1  9a3aa92c-962f-43cf-852a-acf6d72a0fef  label                   20210325T1530Z_1D_NL_EQ_001.xml  97c347d7-92f6-4944-9e34-738df43e9561
2  d039b6c7-fd32-4987-8bc7-53dc5067af0f   Type                                      NamespaceMap  97c347d7-92f6-4944-9e34-738df43e9561
3  d039b6c7-fd32-4987-8bc7-53dc5067af0f    cim                        http://iec.ch/TC57/CIM100#  97c347d7-92f6-4944-9e34-738df43e9561
4  d039b6c7-fd32-4987-8bc7-53dc5067af0f     md  http://iec.ch/TC57/61970-552/ModelDescription/1#  97c347d7-92f6-4944-9e34-738df43e9561
PhaseTapChangerTablePoint 34
RatioTapChangerTablePoint 31
Terminal 30
CurrentLimit 26
OperationalLimitSet 16
PowerTransformerEnd 8
EquivalentInjection 5
ACLineSegment 5
TieFlow 5
Synchronous

### Question 1

1. What is the total production capacity of the generators in the model?
    - Max production: 1500 kW
    - Nominal: 1440 kW
2. What is each generating unit power factor?
    - 0.9 for each one.


In [17]:
df_gen = df.type_tableview("GeneratingUnit") # object type
# format(df_gen)
format(df_gen[["GeneratingUnit.maxOperatingP",
               "GeneratingUnit.nominalP"]])

total_capacity_gen = int(df_gen["GeneratingUnit.maxOperatingP"].sum())
print(f"Max total production capacity: {total_capacity_gen} kW")

┌──────────────────────────────────────┬────────────────────────────────┬───────────────────────────┐
│ ID                                   │   GeneratingUnit.maxOperatingP │   GeneratingUnit.nominalP │
├──────────────────────────────────────┼────────────────────────────────┼───────────────────────────┤
│ 049438a6-780a-44fe-a788-ebe385d98e25 │                           1000 │                       990 │
├──────────────────────────────────────┼────────────────────────────────┼───────────────────────────┤
│ b850063d-eae7-4675-bc98-4642d3076783 │                            250 │                       225 │
├──────────────────────────────────────┼────────────────────────────────┼───────────────────────────┤
│ ca80ee09-3bed-4884-bc28-6dc89d067289 │                            250 │                       225 │
└──────────────────────────────────────┴────────────────────────────────┴───────────────────────────┘ 

Max total production capacity: 1500 kW


In [18]:
df_power_factor = df.type_tableview("SynchronousMachine") # object type
# format(df_power_factor) # For inspecting the whole dataframe

print("Power factors of each generator:")
format(df_power_factor[["RotatingMachine.ratedPowerFactor","IdentifiedObject.name"]]) # Exctract only relevant columns

Power factors of each generator:
┌──────────────────────────────────────┬────────────────────────────────────┬─────────────────────────┐
│ ID                                   │   RotatingMachine.ratedPowerFactor │ IdentifiedObject.name   │
├──────────────────────────────────────┼────────────────────────────────────┼─────────────────────────┤
│ 1dc9afba-23b5-41a0-8540-b479ed8baf4b │                                0.9 │ NL-G3                   │
├──────────────────────────────────────┼────────────────────────────────────┼─────────────────────────┤
│ 2844585c-0d35-488d-a449-685bcd57afbf │                                0.9 │ NL-G2                   │
├──────────────────────────────────────┼────────────────────────────────────┼─────────────────────────┤
│ 9c3b8f97-7972-477d-9dc8-87365cc0ad0e │                                0.9 │ NL-G1                   │
└──────────────────────────────────────┴────────────────────────────────────┴─────────────────────────┘ 



### Question 2
1. What kind of regulation control generator NL-G1 has? Why this and do you know any other kinds?
    - Voltage regulated, this works by adjusting the reactive power provided
    - Examples of other kind of control are: active power, reactive power and power factor.


In [19]:
type_regulation = df.type_tableview("RegulatingControl") # All regulating control types
format(type_regulation[["IdentifiedObject.name",
                        "RegulatingControl.mode"]])

type_regulation = type_regulation[type_regulation["IdentifiedObject.name"] == "NL-G1"] # NL-G1 row only
control = type_regulation["RegulatingControl.mode"] # Control mode
print("Regulation control mode:" , str(control.values[0]))

┌──────────────────────────────────────┬─────────────────────────┬───────────────────────────────────┐
│ ID                                   │ IdentifiedObject.name   │ RegulatingControl.mode            │
├──────────────────────────────────────┼─────────────────────────┼───────────────────────────────────┤
│ 04f338d3-3c0d-433f-a77b-e36dd256f0f0 │ NL-G1                   │ RegulatingControlModeKind.voltage │
├──────────────────────────────────────┼─────────────────────────┼───────────────────────────────────┤
│ 613dfeb3-0161-4cec-aaa9-ccddd885771b │ NL-S1                   │ RegulatingControlModeKind.voltage │
└──────────────────────────────────────┴─────────────────────────┴───────────────────────────────────┘ 

Regulation control mode: RegulatingControlModeKind.voltage


### Question 3

1. What are the nominal voltages of the windings of the transformer NL_TR2_2 (ID: _2184f365-8cd5-4b5d-8a28-9d68603bb6a4)? What type of transformer is this in the grid? 

While investigating the voltages of the windings, it seems that all the other windings except both of the second windings of NL-TR2_1/2, were missing the "baseVoltage" element definition, only the "15.75 kV" level was defined. The voltage values are still listed in the "PowerTransformerEnd" element but I'm guessing they should each have their separate definition also.


In [20]:
NL_TR2_2_name = "NL_TR2_2"
windings = df.type_tableview("PowerTransformerEnd") # All windings
format(windings[["IdentifiedObject.name",
                 "PowerTransformerEnd.ratedU"]])

all_id = windings["TransformerEnd.BaseVoltage"].unique()

for id in all_id:
    print("============ BASE VOLTAGE =================")
    voltage_limit = parser.id_tableview(df,id)
    try:
        format(voltage_limit[["BaseVoltage.nominalVoltage"]])
    except:
        print("No data")

No data available for                                      ID
0  a7f1d8de-d658-428a-821b-3a5ae5965fd1


No data available for                                      ID
0  abb2348a-aa10-446f-9f5d-49f93f211534
No data available for                                      ID
0  597e44dc-2a6e-4c62-82f3-f82cc46e0e14


┌──────────────────────────────────────┬─────────────────────────┬──────────────────────────────┐
│ ID                                   │ IdentifiedObject.name   │   PowerTransformerEnd.ratedU │
├──────────────────────────────────────┼─────────────────────────┼──────────────────────────────┤
│ 0dbed103-fc51-4df4-a6fa-0dee4c57f3a2 │ NL_TR2_2                │                       220    │
├──────────────────────────────────────┼─────────────────────────┼──────────────────────────────┤
│ 0dbed103-fc51-4df4-a6fa-0dee4c57f3a3 │ NL_TR2_3                │                       220    │
├──────────────────────────────────────┼─────────────────────────┼──────────────────────────────┤
│ 0dbed103-fc51-4df4-a6fa-0dee4c57f3a4 │ NL_TR2_4                │                       220    │
├──────────────────────────────────────┼─────────────────────────┼──────────────────────────────┤
│ 0dec3883-a2b6-4353-83cc-d4dd5df513a6 │ NL-TR2_1                │                       220    │
├───────────────────

#### Voltage
- High end: 220 kV
- Low end: 15.75 kV

#### Type of transformer
- This is a two winding step-up transformer

In [21]:
# Only NL_TR2_2 windings
windings = windings[windings["IdentifiedObject.name"]==NL_TR2_2_name]
# format(windings[["IdentifiedObject.name",
#                  "PowerTransformerEnd.ratedU"]])

base_id = windings["TransformerEnd.BaseVoltage"]
# baseVoltage_value_low = parser.id_tableview(df,base_id.iloc[0])
print("\n")
print("High end voltage: missing element definition, should be 220 kV")

baseVoltage_value_low = parser.id_tableview(df,base_id.iloc[1])
print(f"Low end voltage: {baseVoltage_value_low["BaseVoltage.nominalVoltage"].iloc[0]} kV")





High end voltage: missing element definition, should be 220 kV
Low end voltage: 15.75 kV


### Question 4

4.	What is permanently allowed limit for line segment NL-Line_5 (ID: _e8acf6b6-99cb-45ad-b8dc-16c7866a4ddc) and temporarily allowed. What is difference between those limits. List other types of limits in the grid.

During the manual investigation of the file, it seems there isn't a ID connection for the line limit and line segment element, the only connection is by name and it's in the description of the "OperationalLimitSet" types. Of course the limits can be seen from the file, they are right below, but a workflow of finding them manually is demonstrated below:


#### Taking a look at all the different limits



In [22]:
operation_limit = df.type_tableview("OperationalLimitSet")
operation_limit

KEY,IdentifiedObject.description,IdentifiedObject.mRID,IdentifiedObject.name,OperationalLimitSet.Terminal,Type
ID,,,,,
03943678-b63e-44d8-bc7d-9461da496268,Limit-Ratings for branch NL-Line_4 at Port 2,03943678-b63e-44d8-bc7d-9461da496268,Limits at Port 2,57979d14-d1d8-4311-ae53-aa8890423885,OperationalLimitSet
125eeb58-1c45-41b2-a981-7d5993b40cda,Limit-Ratings for branch NL-Line_1 at Port 2,125eeb58-1c45-41b2-a981-7d5993b40cda,Limits at Port 2,6b1cd30c-19ba-44e1-9447-d01db6b1ef9d,OperationalLimitSet
13ee714f-6df4-4039-8519-5cf4e84ab20d,Limit-Ratings for branch NL-Line_2 at Port 1,13ee714f-6df4-4039-8519-5cf4e84ab20d,Limits at Port 1,c557146e-dfc4-4020-9738-d592b188338b,OperationalLimitSet
14e9e317-a168-44a2-82ef-7c76190108a2,Limit-Ratings for branch NL-Line_3 at Port 2,14e9e317-a168-44a2-82ef-7c76190108a2,Limits at Port 2,6aecb9ba-5835-4b70-89bc-96d687e45779,OperationalLimitSet
261d13c4-1527-42a5-ad0d-a3b817971166,Limit-Ratings for branch NL_TR2_2 at Port 2,261d13c4-1527-42a5-ad0d-a3b817971166,Limits at Port 2,31bf04d0-7e34-465b-95a4-0604700325cf,OperationalLimitSet
37cc4210-71c0-4328-a86b-df0f65e76ef4,Limit-Ratings for branch NL-Line_4 at Port 1,37cc4210-71c0-4328-a86b-df0f65e76ef4,Limits at Port 1,653bc4b8-518b-4adc-9d68-012fb641fb1d,OperationalLimitSet
79a1fb3a-a66a-468c-8c78-c25714c73eba,Limit-Ratings for branch NL-Line_1 at Port 1,79a1fb3a-a66a-468c-8c78-c25714c73eba,Limits at Port 1,357e3e14-c38e-4a7e-9c95-b3dbe158f5f3,OperationalLimitSet
7cf4108d-d355-48f5-8556-d42c37a8e2c8,Limit-Ratings for branch NL-Line_2 at Port 2,7cf4108d-d355-48f5-8556-d42c37a8e2c8,Limits at Port 2,8f308117-8bbd-4798-b145-4e78f7d049e7,OperationalLimitSet
83f719ad-2083-44ea-9ebc-9f456c114bdb,Limit-Ratings for branch NL-TR2_1 at Port 1,83f719ad-2083-44ea-9ebc-9f456c114bdb,Limits at Port 1,e3e0c496-5837-4f0f-a596-cc421940f73f,OperationalLimitSet


#### Extract the limits containing "NL-Line_5"

In [23]:
operation_limit = operation_limit[operation_limit["IdentifiedObject.description"].str.contains("NL-Line_5")]
operation_limit

KEY,IdentifiedObject.description,IdentifiedObject.mRID,IdentifiedObject.name,OperationalLimitSet.Terminal,Type
ID,,,,,
a2c3d48f-d768-4e1e-8841-6e9c4913f11f,Limit-Ratings for branch NL-Line_5 at Port 1,a2c3d48f-d768-4e1e-8841-6e9c4913f11f,Limits at Port 1,757d4f50-707b-47a0-891c-cbaefd649631,OperationalLimitSet
b4d8f5e2-7b7a-45eb-b65a-252843d6acf4,Limit-Ratings for branch NL-Line_5 at Port 2,b4d8f5e2-7b7a-45eb-b65a-252843d6acf4,Limits at Port 2,ae588863-b154-451d-978a-7ab08ac50fb6,OperationalLimitSet


#### Print the limit elements for both ports

Here it can be seen that the permanent (PATL) and temporary (TAPL) line limits are 1876 A and 500 A respectively, for both ports. This of course does not make sense since the temporary limit is supposed to be the higher one, so this should be one of the errors in the model and actually the limits should be in reverse.

In [24]:
operational_ids = operation_limit.index
for id in operational_ids:
    limits = df.references_to(id)
    print(limits["VALUE"])

181                             OperationalLimitSet
182                                Limits at Port 1
183    Limit-Ratings for branch NL-Line_5 at Port 1
184            757d4f50-707b-47a0-891c-cbaefd649631
185            a2c3d48f-d768-4e1e-8841-6e9c4913f11f
                           ...                     
12                                             TATL
13             a2c3d48f-d768-4e1e-8841-6e9c4913f11f
14             bf2a4896-2e92-465b-b5f9-b033993a31c8
15                                              500
16             08ad0781-b65c-4f2e-b5ba-00d58bcf2e58
Name: VALUE, Length: 22, dtype: string


198                             OperationalLimitSet
199                                Limits at Port 2
200    Limit-Ratings for branch NL-Line_5 at Port 2
201            ae588863-b154-451d-978a-7ab08ac50fb6
202            b4d8f5e2-7b7a-45eb-b65a-252843d6acf4
                           ...                     
12                                             TATL
13             b4d8f5e2-7b7a-45eb-b65a-252843d6acf4
14             bf2a4896-2e92-465b-b5f9-b033993a31c8
15                                              500
16             86764bf6-6472-4ebe-9d6b-bcceffe66dad
Name: VALUE, Length: 22, dtype: string


#### Other limit elements in the model

Using the same workflow, the other limits could be inspected, here for example all the limit values are listed.

In [25]:
df.type_tableview("CurrentLimit")

KEY,CurrentLimit.normalValue,IdentifiedObject.mRID,IdentifiedObject.name,OperationalLimit.OperationalLimitSet,OperationalLimit.OperationalLimitType,Type
ID,,,,,,
03287f0c-5562-4533-be7f-2fb40e1888f5,2000.0,03287f0c-5562-4533-be7f-2fb40e1888f5,PATL,14e9e317-a168-44a2-82ef-7c76190108a2,1aee0921-7cbe-481c-9078-daf66ab2a950,CurrentLimit
08ad0781-b65c-4f2e-b5ba-00d58bcf2e58,500.0,08ad0781-b65c-4f2e-b5ba-00d58bcf2e58,TATL,a2c3d48f-d768-4e1e-8841-6e9c4913f11f,bf2a4896-2e92-465b-b5f9-b033993a31c8,CurrentLimit
12f7803a-c65b-4a14-8c0d-7e52becb5772,46188.04,12f7803a-c65b-4a14-8c0d-7e52becb5772,PATL,261d13c4-1527-42a5-ad0d-a3b817971166,1aee0921-7cbe-481c-9078-daf66ab2a950,CurrentLimit
1b65a297-4735-499f-8fe8-f08ea01b291f,1233.9,1b65a297-4735-499f-8fe8-f08ea01b291f,PATL,79a1fb3a-a66a-468c-8c78-c25714c73eba,1aee0921-7cbe-481c-9078-daf66ab2a950,CurrentLimit
26069582-0f76-47fb-8bc0-44c99fb08887,1233.9,26069582-0f76-47fb-8bc0-44c99fb08887,PATL,125eeb58-1c45-41b2-a981-7d5993b40cda,1aee0921-7cbe-481c-9078-daf66ab2a950,CurrentLimit
...,...,...,...,...,...,...
b780341b-7b90-498f-ab63-9deb16eff206,500.0,b780341b-7b90-498f-ab63-9deb16eff206,TATL,37cc4210-71c0-4328-a86b-df0f65e76ef4,bf2a4896-2e92-465b-b5f9-b033993a31c8,CurrentLimit
c743cfb3-8279-4721-9eee-12020dd89f64,2000.0,c743cfb3-8279-4721-9eee-12020dd89f64,PATL,9a03e8ee-e832-43c8-b38e-0d43f8ef7355,1aee0921-7cbe-481c-9078-daf66ab2a950,CurrentLimit
d7422ce6-a13d-41ca-8111-9462b2e789dc,500.0,d7422ce6-a13d-41ca-8111-9462b2e789dc,TATL,7cf4108d-d355-48f5-8556-d42c37a8e2c8,bf2a4896-2e92-465b-b5f9-b033993a31c8,CurrentLimit


### Question 5

5.	Which generator is set as slack in the model? Why does model need slack node?
- My conclusion from the available information is that the slack generator could not be determined from this model. My research yielded that either the "AngleRefTopologicalNode" or "referencePriority" is needed for this evaluation, both of which are missing from this model. Maybe there is another way to determine this from the model but I could not indentify it.
- The slack node is used as a reference point for the power system and to balance out active and reactive power in the system.

In [26]:
df_power_factor # reusing the variable from Q1

KEY,Equipment.EquipmentContainer,Equipment.aggregate,IdentifiedObject.description,IdentifiedObject.mRID,...,SynchronousMachine.voltageRegulationRange,SynchronousMachine.x0,SynchronousMachine.x2,Type
ID,,,,,,,,,
1dc9afba-23b5-41a0-8540-b479ed8baf4b,8d8a82ba-b5b0-4e94-861a-192af055f2b8,false,Machine,1dc9afba-23b5-41a0-8540-b479ed8baf4b,...,0,0.13,0.17,SynchronousMachine
2844585c-0d35-488d-a449-685bcd57afbf,8d8a82ba-b5b0-4e94-861a-192af055f2b8,false,Machine,2844585c-0d35-488d-a449-685bcd57afbf,...,0,0.1,0.16,SynchronousMachine
9c3b8f97-7972-477d-9dc8-87365cc0ad0e,8d8a82ba-b5b0-4e94-861a-192af055f2b8,false,Machine,9c3b8f97-7972-477d-9dc8-87365cc0ad0e,...,0,0.11,0.18,SynchronousMachine


### Question 6 
1.	Find mistakes in the model (both semantic, power system related and logical errors are present)

#### Semantic
To get the file running first some xml definitions needed to be fixed
- Missmatch of some opening and closing tags
![alt text](../images/tags.png)
- Line 678, 701 and 724 had the same ID for the "PowerTransformerEnd", which is not allowed, changed the ID endings so every ID was unique.

#### Power system/logic related
- Missing "baseVoltage" elements for all the windings
- Swapped PATL/TATL values
- Missing slack generator definition!?

